In [13]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly,KNNBasic,NormalPredictor,SlopeOne,accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
import pandas as pd

In [4]:
# 数据读取
reader=Reader(line_format='user item rating timestamp',sep=',',skip_lines=1)
data=Dataset.load_from_file('./ratings.csv',reader=reader)
train_set=data.build_full_trainset()

## BaselineOnly算法

+ BaselineOnly 算法是根据Factor in the neighbors:scalable and accurate collaborative filtering 所得到，算法发现传统CF方法的一些问题：
 + 不同 item 的评分不同
 + 不同用户的评分也不同
 + 评分随着时间一直在变化

Baseline model:
$$ b_{ui}=u+b_u+b_i $$, 其中u是平均得分，$b_u$是用户的偏置，$b_i$是item的偏置，等价于求下面的极值问题：  
$$ minimize \sum_{(u,i)}{(r_{ui}-u-b_i)^2}+\lambda [\sum_u{b_u}^2+\sum_i{b_i}^2] $$

Baseline模型虽然简单， 但其中其实已经包含了用户和item的个性化信息。在面对大规模数据时， 简单算法能够减少大量的计算时间

In [5]:
algo=BaselineOnly()
algo.fit(train_set)

Estimating biases using als...


In [6]:
uid=str(196)
iid=str(332)
pred=algo.predict(uid,iid,r_ui=4,verbose=True)

user: 196        item: 332        r_ui = 4.00   est = 3.11   {'was_impossible': False}


In [11]:
kf=KFold(n_splits=3)
for trainset,testset in kf.split(data):
    algo.fit(trainset)
    predictions=algo.test(testset)
    #计算RMSE
    accuracy.rmse(predictions,verbose=True)

Estimating biases using als...
RMSE: 0.8653
Estimating biases using als...
RMSE: 0.8649
Estimating biases using als...
RMSE: 0.8678


# slope one 算法思想

Slope One算法是基于不同物品之间的评分差的线性算法，预测用户对物品评分的个性化算法。主要两步：
+ Step1:计算物品之间的评分差的均值，记为物品间的评分偏差(两物品同时被评分)；
+ Step2:根据物品间的评分偏差和用户的历史评分，预测用户对未评分的物品的评分;
+ Step3:将预测评分排序，取topN对应的物品推荐给用户。

In [14]:
# 使用SlopeOne算法
algo = SlopeOne()
algo.fit(train_set)
# 对指定用户和商品进行评分预测
uid = str(196) 
iid = str(302) 
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.32   {'was_impossible': False}


In [15]:
kf=KFold(n_splits=3)
for trainset,testset in kf.split(data):
    algo.fit(trainset)
    predictions=algo.test(testset)
    #计算RMSE
    accuracy.rmse(predictions,verbose=True)

RMSE: 0.8675
RMSE: 0.8678
RMSE: 0.8679
